In [ ]:
import tensorflow as tf
from tensorflow.python.saved_model import tag_constants
from compression import face_classifier_reduced

In [ ]:
with tf.Session() as sess:
    # load model
    tf.saved_model.loader.load(sess, [tag_constants.SERVING], './saved_model')
    
    # collect layers
    layers = {item.name: item.eval() for item in tf.get_collection(key=tf.GraphKeys.TRAINABLE_VARIABLES)}

    # rebuild model with SVD decomposition of dense layers
    images_batch = tf.placeholder(tf.float32, [None, 218, 178, 3])
    labels = tf.placeholder(tf.int32, [None])
    classifier = face_classifier_reduced(sess, images_batch, layers, 1)

    # save compressed model
    inputs = {
        "images_batch": images_batch,
        "labels": labels
    }
    outputs = {"predictions": classifier}
    tf.saved_model.simple_save(sess, './compressed', inputs, outputs)